# Part 1
(This is the working notebook)

In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.naive_bayes import MultinomialNB
from sklearn.impute import SimpleImputer

from utilities import *

In [2]:
def preprocess(df):
    # Extract unique countries in the df
    unique_values = set()
    df['countries_in_family'].apply(lambda x: unique_values.update(x.strip("[]").replace("'", "").split())) 

    # Create new columns for each unique value
    for value in unique_values:
        # each country has a column (1 if the patent belongs to the country 0 otherwise)
        df[value] = df['countries_in_family'].apply(lambda x: 1 if value in x else 0)

    df = df[df.abstract.notna()].copy()  # drop all samples without abstract

    # Encode company names
    df['company_name_encoded'] = df.company_name.astype('category').cat.codes  # encode companies

    # Remove non-numeric columns
    df_columns_dropped = df.drop(['publication_number', 'company_name', 'countries_in_family', 'publn_nr','primary_cpc'], axis=1)

    # f0_ has the same value as commercialization, the other two shouldn't be used
    df_columns_dropped = df_columns_dropped.drop(['f0_', 'centrality', 'similarity'], axis=1)

    # Remove text as I can't compute min and max on it
    text = df_columns_dropped[['abstract', 'description_text']]  # putting them aside for later
    df_columns_dropped.drop(['abstract', 'description_text'], axis=1, inplace=True)

    df_no_missing = df_columns_dropped.fillna(df_columns_dropped.mean()).copy()

    # Extracting what we'll try to predict
    y = df_no_missing['commercialized']
    df_no_missing.drop('commercialized', axis=1, inplace=True)

    # Dropping columns where all the values are the same (min = max)
    min_eq_max = df_no_missing.columns[df_no_missing.min() == df_no_missing.max()].to_list()
    df_clean = df_no_missing.drop(min_eq_max, axis=1)

    X_train, X_test, y_train, y_test = train_test_split(df_clean, y, test_size=0.20, random_state=42)

    # Rescale
    scaler = StandardScaler()
    scaler.fit(X_train)
    X_train_scaled = scaler.transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # # Putting text back in
    # df_clean[['abstract', 'description_text']] = text 

    X_train, X_test, y_train, y_test = train_test_split(df_clean, y, test_size=0.20, random_state=42) 

    # # Same vectorizer applied to training and testing for abstract
    # vectorizer = TfidfVectorizer(max_features=1000)  # Adjust 'max_features' as needed
    # X_train_ab = encode_text_colum(X_train, 'abstract', vectorizer)
    # X_test_ab = encode_text_colum(X_test, 'abstract', vectorizer)

    # # Same vectorizer applied to training and testing for description_text
    # vectorizer = TfidfVectorizer(max_features=1000)  # Adjust 'max_features' as needed
    # X_train_de = encode_text_colum(X_train_ab, 'description_text', vectorizer)
    # X_test_de = encode_text_colum(X_test_ab, 'description_text', vectorizer)

    # return X_train_de, X_test_de, y_train, y_test
    return df_clean, y, X_train, X_test, y_train, y_test


In [3]:
df = pd.read_csv('data/modelready_220423.csv')

In [26]:
from sklearn.preprocessing import MinMaxScaler

# Apply the preprocessing and encoding function
df_clean, y, X_train, X_test, y_train, y_test = preprocess(df)

In [27]:
print(f"There aren't any NaN values in X_train_de:\n>> {X_train.isna().sum().sum() == 0}")

There aren't any NaN values in X_train_de:
>> True


In [28]:
import numpy as np
from sklearn.naive_bayes import MultinomialNB

def train_naive_bayes_model(X_train, y_train):
    # Create and fit the Naive Bayes model
    nb_classifier = MultinomialNB()
    nb_classifier.fit(X_train, y_train)

    return nb_classifier

def evaluate_model(nb_classifier, X_test, y_test):
    # Make predictions using the trained model
    y_pred = nb_classifier.predict(X_test)

    # Calculate the accuracy
    accuracy = np.mean(y_pred == y_test)
    print("Accuracy:", accuracy)

# Rescaling
scaler = MinMaxScaler()
scaler.fit(X_train)
X_train_array = scaler.transform(X_train.values)
X_test_array = scaler.transform(X_test.values)

# Load the preprocessed data
X_train = X_train_array
X_test = X_test_array

# Train the Naive Bayes model
nb_classifier = train_naive_bayes_model(X_train, y_train)

# Evaluate the model performance
evaluate_model(nb_classifier, X_test, y_test)


/Users/valerio/Library/Python/3.11/lib/python/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/Users/valerio/Library/Python/3.11/lib/python/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


Accuracy: 0.8186311077955987


# Part 2

In [29]:
def modify_df(df, cols_to_drop):
    df_out = df.copy()
    df_out = df_out.drop(cols_to_drop, axis=1)
    return df_out

In [30]:
any_nan = df_clean.isna().any().any()
print("Any NaN values in the DataFrame:", any_nan)

Any NaN values in the DataFrame: False


In [31]:
df_clean['backward_citations_app']

0        140
1        203
2         69
3        162
4        208
        ... 
63342     75
63343     15
63346     64
63347      1
63348     50
Name: backward_citations_app, Length: 53616, dtype: int64

In [32]:
to_drop = ['backward_citations_app', 'backward_citations_exa'] 
modify_df(df_clean, to_drop).shape

(53616, 842)

In [34]:
import numpy as np
import pandas as pd
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split

# Load your preprocessed data (replace X_train_array, X_test_array, y_train, y_test with your actual data)
# For this example, I'll assume you have them already defined.
# X_train_array, X_test_array, y_train, y_test = ...

# Function to train Naive Bayes model
def train_naive_bayes_model(X_train, y_train):
    nb_classifier = MultinomialNB()
    nb_classifier.fit(X_train, y_train)
    return nb_classifier

# Function to evaluate Naive Bayes model
def evaluate_model(nb_classifier, X_test, y_test):
    y_pred = nb_classifier.predict(X_test)
    accuracy = np.mean(y_pred == y_test)
    print("Accuracy:", accuracy)

# Function to modify the dataframe by dropping specified columns
def modify_df(df, cols_to_drop):
    df_out = df.drop(cols_to_drop, axis=1)
    return df_out

# Step 1: Group Features Based on Correlation
correlation_matrix = df_clean.corr().abs()

# Set a correlation threshold
correlation_threshold = 0.8

# Find groups of correlated features
feature_groups = []
while len(correlation_matrix.columns) > 0:
    feature = correlation_matrix.columns[0]
    correlated_features = list(correlation_matrix[feature][correlation_matrix[feature] > correlation_threshold].index)
    feature_groups.append([feature] + correlated_features)
    correlation_matrix = correlation_matrix.drop(correlated_features, axis=1)
    correlation_matrix = correlation_matrix.drop(feature, axis=0)

# Step 2: Iterate Over Feature Groups and Train Naive Bayes
for group in feature_groups:
    # Modify the dataframe by dropping the current feature group
    df_modified = modify_df(df_clean, group)

    # Split the modified data
    X_train_mod, X_test_mod, y_train_mod, y_test_mod = train_test_split(df_modified, y, test_size=0.20, random_state=42)

    # Rescaling the modified data
    scaler = MinMaxScaler()
    scaler.fit(X_train_mod)
    X_train_array_mod = scaler.transform(X_train_mod.values)
    X_test_array_mod = scaler.transform(X_test_mod.values)

    # Load the preprocessed data
    X_train_mod = X_train_array_mod
    X_test_mod = X_test_array_mod

    # Train Naive Bayes model on modified data
    nb_classifier_mod = train_naive_bayes_model(X_train_mod, y_train_mod)

    # Evaluate the model performance
    print(f"\nPerformance after removing features {group}:")
    evaluate_model(nb_classifier_mod, X_test_mod, y_test_mod)


/Users/valerio/Library/Python/3.11/lib/python/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/Users/valerio/Library/Python/3.11/lib/python/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(



Performance after removing features ['vpm_patent_score', 'vpm_patent_score']:
Accuracy: 0.7026296158149944


/Users/valerio/Library/Python/3.11/lib/python/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/Users/valerio/Library/Python/3.11/lib/python/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(



Performance after removing features ['backward_citations_app', 'backward_citations_app']:
Accuracy: 0.8179783662812383


/Users/valerio/Library/Python/3.11/lib/python/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/Users/valerio/Library/Python/3.11/lib/python/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


...

/Users/valerio/Library/Python/3.11/lib/python/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/Users/valerio/Library/Python/3.11/lib/python/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(



Performance after removing features ['dummy_cpc_E05B', 'dummy_cpc_E05B']:
Accuracy: 0.8186311077955987


/Users/valerio/Library/Python/3.11/lib/python/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/Users/valerio/Library/Python/3.11/lib/python/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(



Performance after removing features ['dummy_cpc_E05C', 'dummy_cpc_E05C']:
Accuracy: 0.8186311077955987


KeyboardInterrupt: 